# Abstractive summaries - Train DistilBART on TWEETSUMM dataset

In [1]:
import json, re
from huggingface_hub import notebook_login
import pandas as pd
import numpy as np
import os, time, datetime

try:
    from datasets import load_dataset
except:
    !pip install datasets
    from datasets import load_dataset

# try:
#     import accelerate
# except:
#     !pip install -U 'accelerate==0.27.2'
#     import accelerate


# import transformers
# from transformers import AutoTokenizer, DataCollatorForSeq2Seq, pipeline, set_seed
# from transformers import AutoModelForSeq2SeqLM, Seq2SeqTrainingArguments, Seq2SeqTrainer
# from transformers import GenerationConfig

try:
  import wandb
except:
  !pip install wandb
  import wandb
from transformers import pipeline, set_seed
# print(transformers.__version__, accelerate.__version__)


c:\Python312\Lib\site-packages\transformers\utils\generic.py:441: UserWarning: torch.utils._pytree._register_pytree_node is deprecated. Please use torch.utils._pytree.register_pytree_node instead.
  _torch_pytree._register_pytree_node(
c:\Python312\Lib\site-packages\transformers\utils\generic.py:309: UserWarning: torch.utils._pytree._register_pytree_node is deprecated. Please use torch.utils._pytree.register_pytree_node instead.
  _torch_pytree._register_pytree_node(
c:\Python312\Lib\site-packages\transformers\utils\generic.py:309: UserWarning: torch.utils._pytree._register_pytree_node is deprecated. Please use torch.utils._pytree.register_pytree_node instead.
  _torch_pytree._register_pytree_node(


In [2]:
# try:
#   import transformers
# except:
#   !pip install -U transformers[torch]
#   import transformers

In [3]:
# for x in dir(transformers):
#     if "torch" in x:
#         print(x)
        
# print(transformers.is_tf_available())

In [4]:
ds_dir = ""
try:
    HF_TOKEN =  os.environ['HF_TOKEN']
except:
    HF_TOKEN = ""

if 'google.colab' in str(get_ipython()):
  print("Running on Colab")
  from google.colab import drive, userdata
  drive.mount('/content/drive')
  HF_TOKEN = userdata.get('HF_TOKEN')
elif os.environ.get('KAGGLE_KERNEL_RUN_TYPE'):
  from kaggle_secrets import UserSecretsClient
  print("Running on Kaggle")
  ds_dir = "/kaggle/input/tweet-data-2106-1512/"
  user_secrets = UserSecretsClient()
  HF_TOKEN = user_secrets.get_secret("HF_TOKEN")
  WANDB_API_KEY = user_secrets.get_secret("WANDB_API_KEY")
  os.environ['WANDB_API_KEY'] = WANDB_API_KEY


In [5]:
set_seed(17)

os.environ["WANDB_PROJECT"] = "aiml-thesis-test"
os.environ["WANDB_LOG_MODEL"] = "checkpoint"

wandb.init(settings=wandb.Settings(start_method="thread"))

Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit:wandb: Appending key for api.wandb.ai to your netrc file: C:\Users\gracz\_netrc


In [6]:
from huggingface_hub import login
login(token=HF_TOKEN)

The token has not been saved to the git credentials helper. Pass `add_to_git_credential=True` in this function directly or `--add-to-git-credential` if using via `huggingface-cli` if you want to set the git credential as well.
Token is valid (permission: write).
Your token has been saved to C:\Users\gracz\.cache\huggingface\token
Login successful


## Load data

In [7]:
!cd /kaggle/input/ && ls && cd data && ls 

The system cannot find the path specified.


In [10]:
test_df_temp = pd.read_feather(ds_dir + "data/train_dial_abs_noex_noco_2006.feather")
test_df_temp.drop(columns=['index', 'company'], inplace=True)

In [11]:
# lengths_train_summaries = [len(row) for row in tokenized_tweetsumm_abs['train']['input_ids']]
# print("Abstractive summaries training lengths[mean,max,min]:", np.mean(lengths_train_summaries), np.max(lengths_train_summaries), np.min(lengths_train_summaries))

In [12]:
from datasets import Dataset, DatasetDict

tweetsum_test_abs = DatasetDict(
 {
 'test': Dataset.from_pandas(test_df_temp),
 }
)

In [16]:
tweetsum_test_abs['test'][10]

{'dialogue': "<USER> Bought these biscuits a couple of weeks ago, only just opened and looked at the best before date... @49975 <SYSTEM> @393926 We can't see a picture of the best before date or biscuits attached to your tweets here. Please can you tweet or DM it to us again? Thanks. <URL> <USER> @marksandspencer  <URL> @marksandspencer  <URL> @marksandspencer  <URL> <SYSTEM> @393926 We'd certainly like to take a closer look into this. Please DM us a picture of your full receipt. <URL> <USER> @marksandspencer Don’t have a receipt as they were bought for my grandparents 3 weeks ago and have only just realised the date on them @marksandspencer The biscuits were purchased at the Marks and Spencer’s Store at the Ricoh Arena, Coventry <SYSTEM> @393926 Did yo use a sparks card on your transaction, Cian? <USER> @marksandspencer Yes <SYSTEM> @393926 Hi Cian. I'm really sorry to see this, especially when it was such a lovely gesture too! No worries though - we got your back ;) 1/3 @393926 Is th

In [17]:
# Source: https://huggingface.co/docs/transformers/en/tasks/summarization

def preprocess_function(examples):
  prefix = "summarize: "
  inputs = [prefix + dial for dial in examples["dialogue"]]
  model_inputs = tokenizer(inputs, max_length=512, truncation=True) # same params as tweetsumm paper
  labels = tokenizer(text_target=examples["summary"], max_length=80, truncation=True)
  model_inputs["labels"] = labels["input_ids"]
  return model_inputs

In [18]:
checkpoint_bart = "roequitz/trained-distilbart-abs-0807"

In [19]:
bart_tokenizer = AutoTokenizer.from_pretrained(checkpoint_bart)
tokenizer = bart_tokenizer
tokenized_tweetsumm_abs = tweetsum_train_val_abs.map(preprocess_function, batched=True)

NameError: name 'AutoTokenizer' is not defined

In [20]:
# min_ln = (9999, -1, 'train')
# max_ln = (0, -1, 'train')
# mean_ln = 0
# for spl in ('validation','train'):
#     summ_lengths = []
#     for idx, summ in enumerate(tokenized_tweetsumm_abs[spl]['summary']):
#         summ_length = len(str(summ).split())
        
#         min_ln = (min(min_ln[0], summ_length),idx, spl)
#         max_ln = (max(max_ln[0], summ_length),idx, spl)
#         mean_ln += summ_length
all_summ_lengths = []

for spl in ('validation', 'train'):
  summ_lengths = [len(str(summ).split()) for summ in tokenized_tweetsumm_abs[spl]['summary']]
  all_summ_lengths = np.concatenate((all_summ_lengths, summ_lengths), axis=0, dtype=int, casting='unsafe')

min_length = min(all_summ_lengths)
max_length = max(all_summ_lengths)
min_summary = None
max_summary = None

for spl in ('validation', 'train'):
  for idx, summ in enumerate(tokenized_tweetsumm_abs[spl]['summary']):
    summ_length = len(str(summ).split())
    if summ_length == min_length:
      min_summary = summ
    elif summ_length == max_length:
      max_summary = summ
print("Min summary length:", min_length, "Summary:", min_summary)
print("Max summary length:", max_length, "Summary:", max_summary)
# print("Avg summary length:", mean_ln/ (len(tokenized_tweetsumm_abs['train'])+len(tokenized_tweetsumm_abs['validation'])))
# print("Confirmed min summary length:", len(tokenized_tweetsumm_abs[spl][int(np.argmin(summ_lengths))-1]['summary'].split()))


NameError: name 'tokenized_tweetsumm_abs' is not defined

In [ ]:
# data_collator = DataCollatorForSeq2Seq(tokenizer=tokenizer, model=checkpoint_bart)

In [ ]:
print(json.dumps(tokenized_tweetsumm_abs['test'][5], indent=2))

## Setup Training Evaluation

In [21]:
try:
  import evaluate
  rouge = evaluate.load("rouge")
  meteor = evaluate.load("meteor")
  bertscore = evaluate.load("bertscore")
except:
  !pip install evaluate nltk rouge_score bert_score
  !pip install -U nltk
  import evaluate
  rouge = evaluate.load("rouge")
  meteor = evaluate.load("meteor")
  bertscore = evaluate.load("bertscore")

[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\gracz\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\gracz\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package omw-1.4 to
[nltk_data]     C:\Users\gracz\AppData\Roaming\nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


ERROR: Could not install packages due to an OSError: [WinError 2] The system cannot find the file specified: 'C:\\Python312\\Scripts\\bert-score.exe' -> 'C:\\Python312\\Scripts\\bert-score.exe.deleteme'

[notice] A new release of pip is available: 24.0 -> 24.1.2

  Using cached bert_score-0.3.13-py3-none-any.whl.metadata (15 kB)

[notice] To update, run: python.exe -m pip install --upgrade pip

   ---------------------------------------- 0.0/61.1 kB ? eta -:--:--
   ------------- -------------------------- 20.5/61.1 kB 217.9 kB/s eta 0:00:01
   -------------------------- ------------- 41.0/61.1 kB 393.8 kB/s eta 0:00:01
   ---------------------------------------- 61.1/61.1 kB 466.9 kB/s eta 0:00:00


[notice] A new release of pip is available: 24.0 -> 24.1.2

[notice] To update, run: python.exe -m pip install --upgrade pip

[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\gracz\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\gracz\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package omw-1.4 to
[nltk_data]     C:\Users\gracz\AppData\Roaming\nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


In [ ]:
# import numpy as np


# def compute_metrics(eval_pred):
#     predictions, labels = eval_pred
#     decoded_preds = tokenizer.batch_decode(predictions, skip_special_tokens=True)
#     labels = np.where(labels != -100, labels, tokenizer.pad_token_id)
#     decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)
#     # result = rouge.compute(predictions=decoded_preds, references=decoded_labels, use_stemmer=True)
#     result = {
#       'rouge': rouge.compute(predictions=decoded_preds, references=decoded_labels, use_stemmer=True),
#       'bertscore': bertscore.compute(predictions=decoded_preds, references=decoded_labels, lang="en"),
#       'meteor': meteor.compute(predictions=decoded_preds, references=decoded_labels),
#     }
#     prediction_lens = [np.count_nonzero(pred != tokenizer.pad_token_id) for pred in predictions]
#     result["gen_len"] = np.mean(prediction_lens)
#     print(json.dumps(result, indent=2))
#     return {k: round(v, 4) if type(v) != list else v for k, v in result.items()}

In [ ]:
# arrr = [0,1,2,3,4,5,6,7]
# valsss = ['a','b','c','d','e','f','g','h']

# kwkwk = {f"id-{x}": vall for x, vall in enumerate(valsss)}
# origindict = {'alpha':5, **kwkwk}
# print(origindict)

In [22]:
def compute_metrics_abs(eval_pred):
  predictions, labels = eval_pred
  decoded_preds = tokenizer.batch_decode(predictions, skip_special_tokens=True)
  labels = np.where(labels != -100, labels, tokenizer.pad_token_id)
  decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)
  
  # bertscores = bertscore.compute(predictions=decoded_preds, references=decoded_labels, lang="en")
  # np.mean(bertscores)
  # 'rouge': rouge.compute(predictions=decoded_preds, references=decoded_labels, use_stemmer=True,use_aggregator=False),
  # wandb.log({f"losses/loss-{ii}": loss for ii, loss in enumerate(losses)})
  # rouge_scores = {f"rouge/rougerouge-id-{i}": score for i, score in enumerate(rouge.compute(predictions=decoded_preds,
  #                                                                                references=decoded_labels,
  #                                                                                use_stemmer=True,
  #                                                                                use_aggregator=True))}
  # bert_scores = {f"bertscore/bert-id-{i}": score for i, score in enumerate(bertscore.compute(predictions=decoded_preds, references=decoded_labels, lang="en"))},
  rouge_scores = rouge.compute(predictions=decoded_preds, references=decoded_labels, use_stemmer=True, use_aggregator=True)
  bert_scores = bertscore.compute(predictions=decoded_preds, references=decoded_labels, lang="en")
  bert_scores.pop('hashcode')
  result = {
      **{f"rouge/{k}": round(v, 4) for k,v in rouge_scores.items()},
      **{f"bertscore/bertscore-{k}": round(np.mean(v), 4) for k,v in bert_scores.items()},
      'meteor': round(meteor.compute(predictions=decoded_preds, references=decoded_labels)['meteor'], 4),
  }
  #   for k,v in result.items():
  #     print(k, type(v), v)
  # Bug fix source: https://discuss.huggingface.co/t/bug-in-summarization-tutorial/60566/2
  predictions = np.where(predictions != -100, predictions, tokenizer.pad_token_id)
  prediction_lens = [np.count_nonzero(pred != tokenizer.pad_token_id) for pred in predictions]
  result["gen_len"] = np.mean(prediction_lens)
  # print(json.dumps(bertscore.compute(predictions=decoded_preds, references=decoded_labels, lang="en"), indent=2))
  return result # {k: round(v, 4) if type(v) != list else v for k, v in result.items()}

#       'rouge1': round(rouge_scores['rouge1'], 4),
#       'rouge2': round(rouge_scores['rouge2'], 4),
#       'rougeL': round(rouge_scores['rougeL'], 4),
#       'rougeLsum': round(rouge_scores['rougeLsum'], 4),
#       'bertscore/bertscore-precision': np.mean(bertscore.compute(predictions=decoded_preds, references=decoded_labels, lang="en")['precision']),
#       'bertscore/bertscore-recall': np.mean(bertscore.compute(predictions=decoded_preds, references=decoded_labels, lang="en")['recall']),
#       'bertscore/bertscore-f1': np.mean(bertscore.compute(predictions=decoded_preds, references=decoded_labels, lang="en")['f1']),

## Train

In [ ]:
# print(json.dumps(), indent=2)
# blah = bertscore.compute(predictions=['a', 'blue', 'car'], references=['a', 'black', 'car'], lang="en")
# for b,c in blah.items():
#     print(c)
#     print(np.round(sum(c)/len(c), 4))

In [23]:
from transformers import AutoConfig
config = AutoConfig.from_pretrained(checkpoint_bart)
config.max_length = 80
config.min_length = 10
print(config)

c:\Python312\Lib\site-packages\huggingface_hub\file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


config.json:   0%|          | 0.00/1.66k [00:00<?, ?B/s]

c:\Python312\Lib\site-packages\huggingface_hub\file_download.py:157: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\gracz\.cache\huggingface\hub\models--roequitz--trained-distilbart-abs-0807. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to see activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)


BartConfig {
  "_name_or_path": "roequitz/trained-distilbart-abs-0807",
  "_num_labels": 3,
  "activation_dropout": 0.0,
  "activation_function": "gelu",
  "add_bias_logits": false,
  "add_final_layer_norm": false,
  "architectures": [
    "BartForConditionalGeneration"
  ],
  "attention_dropout": 0.0,
  "bos_token_id": 0,
  "classif_dropout": 0.0,
  "classifier_dropout": 0.0,
  "d_model": 1024,
  "decoder_attention_heads": 16,
  "decoder_ffn_dim": 4096,
  "decoder_layerdrop": 0.0,
  "decoder_layers": 6,
  "decoder_start_token_id": 2,
  "dropout": 0.1,
  "early_stopping": true,
  "encoder_attention_heads": 16,
  "encoder_ffn_dim": 4096,
  "encoder_layerdrop": 0.0,
  "encoder_layers": 12,
  "eos_token_id": 2,
  "eos_token_ids": [
    2
  ],
  "extra_pos_embeddings": 2,
  "forced_eos_token_id": 2,
  "gradient_checkpointing": false,
  "id2label": {
    "0": "LABEL_0",
    "1": "LABEL_1",
    "2": "LABEL_2"
  },
  "init_std": 0.02,
  "is_encoder_decoder": true,
  "label2id": {
    "LABEL_0

In [ ]:
# model = AutoModelForSeq2SeqLM.from_pretrained(checkpoint_bart, config=config)

In [ ]:
# os.environ['CUDA_LAUNCH_BLOCKING'] = "1"
# os.environ['TORCH_USE_CUDA_DSA'] = "1"

In [ ]:
# len(tokenized_tweetsumm_abs['train'])

In [24]:
# Print config
print("Tokenizer config:", tokenizer.init_kwargs)
print("Model config:", str(model.config).replace('\n',''))

NameError: name 'tokenizer' is not defined

In [ ]:
# current_time = datetime.datetime.now().strftime("%d%m-%H%M")
# print(current_time)
# run_name_model = f"distilbart-abs-{current_time}"
# wandb.run.name = run_name_model
# wandb.run.save()

# gen_config = GenerationConfig(max_source_length=512,bos_token_id=0)
# gen_config.save_pretrained("roequitz/distilbart-abs-tweetsumm", push_to_hub=True)

# training_args = Seq2SeqTrainingArguments(
#     output_dir=f"trained-distilbart-abs-{current_time[0:4]}",
#     eval_strategy="epoch",
#     logging_strategy="steps",
#     logging_steps=10,
#     learning_rate=3e-5,
#     per_device_train_batch_size=4,
#     per_device_eval_batch_size=4,
#     weight_decay=0.01,
#     save_strategy="epoch",
#     save_total_limit=1,
#     num_train_epochs=6,
#     predict_with_generate=True,
#     fp16=True,
#     generation_max_length=80,
#     generation_config=gen_config,
#     push_to_hub=True,
#     report_to="wandb",
#     run_name=run_name_model
# )
# trainer = Seq2SeqTrainer(
#     model=model,
#     args=training_args,
#     train_dataset=tokenized_tweetsumm_abs["train"],
#     eval_dataset=tokenized_tweetsumm_abs["validation"],
#     tokenizer=tokenizer,
#     data_collator=data_collator,
#     compute_metrics=compute_metrics_abs,
# )

# training_start = time.time()
# trainer.train()
# training_end = time.time()
# print("Time it took for training:", str(datetime.timedelta(seconds=(training_end-training_start))))

In [25]:
from transformers import pipeline
summarizer = pipeline("summarization", model=checkpoint_bart)
summarizer(tweetsum_test_abs['dialogue'])

c:\Python312\Lib\site-packages\huggingface_hub\file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
c:\Python312\Lib\site-packages\transformers\utils\generic.py:309: UserWarning: torch.utils._pytree._register_pytree_node is deprecated. Please use torch.utils._pytree.register_pytree_node instead.
  _torch_pytree._register_pytree_node(


model.safetensors:   0%|          | 0.00/1.22G [00:00<?, ?B/s]

: 